# HMM based PoS tagger
---
The Hidden Markov Model (HMM) is an extension of the Markov process used to model phenomena where the states are hidden or latent, but they emit observations. For instance, in a speech recognition system like a speech-to-text converter, the states represent the actual text words to predict, but they are not directly observable (i.e., the states are hidden). Rather, you only observe the speech (audio) signals corresponding to each word and need to deduce the states using the observations.

Similarly, in POS tagging, you observe the words in a sentence, but the POS tags themselves are hidden. Thus, the POS tagging task can be modeled as a Hidden Markov Model with the hidden states representing POS tags that emit observations, i.e., words.

The hidden states emit observations with a certain probability. Therefore, Hidden Markov Model has emission probabilities, which represent the probability that a particular state emits a given observation. Along with the transition and initial state probabilities, these emission probabilities are used to model HMMs.

The figure below illustrates the emission and transition probabilities for a hidden Markov process with three hidden states and four observations.


![HMM](https://wisdomml.in/wp-content/uploads/2023/04/HMM.png)


### Corpus

**Treebank Tagged Corpus**

The Treebank corpora provide a syntactic parse for each sentence. The NLTK data package includes a **10% sample** of the Penn Treebank (in treebank), as well as the Sinica Treebank (in sinica_treebank).

Reading the Penn Treebank (Wall Street Journal sample):

In [ ]:
!pip install nltk pandas

**Downloading the dataset**

In [6]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     /home/rughimire/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [7]:
from nltk.corpus import treebank
print(treebank.fileids())

['wsj_0001.mrg', 'wsj_0002.mrg', 'wsj_0003.mrg', 'wsj_0004.mrg', 'wsj_0005.mrg', 'wsj_0006.mrg', 'wsj_0007.mrg', 'wsj_0008.mrg', 'wsj_0009.mrg', 'wsj_0010.mrg', 'wsj_0011.mrg', 'wsj_0012.mrg', 'wsj_0013.mrg', 'wsj_0014.mrg', 'wsj_0015.mrg', 'wsj_0016.mrg', 'wsj_0017.mrg', 'wsj_0018.mrg', 'wsj_0019.mrg', 'wsj_0020.mrg', 'wsj_0021.mrg', 'wsj_0022.mrg', 'wsj_0023.mrg', 'wsj_0024.mrg', 'wsj_0025.mrg', 'wsj_0026.mrg', 'wsj_0027.mrg', 'wsj_0028.mrg', 'wsj_0029.mrg', 'wsj_0030.mrg', 'wsj_0031.mrg', 'wsj_0032.mrg', 'wsj_0033.mrg', 'wsj_0034.mrg', 'wsj_0035.mrg', 'wsj_0036.mrg', 'wsj_0037.mrg', 'wsj_0038.mrg', 'wsj_0039.mrg', 'wsj_0040.mrg', 'wsj_0041.mrg', 'wsj_0042.mrg', 'wsj_0043.mrg', 'wsj_0044.mrg', 'wsj_0045.mrg', 'wsj_0046.mrg', 'wsj_0047.mrg', 'wsj_0048.mrg', 'wsj_0049.mrg', 'wsj_0050.mrg', 'wsj_0051.mrg', 'wsj_0052.mrg', 'wsj_0053.mrg', 'wsj_0054.mrg', 'wsj_0055.mrg', 'wsj_0056.mrg', 'wsj_0057.mrg', 'wsj_0058.mrg', 'wsj_0059.mrg', 'wsj_0060.mrg', 'wsj_0061.mrg', 'wsj_0062.mrg', 'wsj_00

In [8]:
print(treebank.words('wsj_0003.mrg'))

['A', 'form', 'of', 'asbestos', 'once', 'used', '*', ...]


In [9]:
print(treebank.tagged_words('wsj_0003.mrg'))

[('A', 'DT'), ('form', 'NN'), ('of', 'IN'), ...]


In [10]:
print(treebank.parsed_sents('wsj_0003.mrg')[0])

(S
  (S-TPC-1
    (NP-SBJ
      (NP (NP (DT A) (NN form)) (PP (IN of) (NP (NN asbestos))))
      (RRC
        (ADVP-TMP (RB once))
        (VP
          (VBN used)
          (NP (-NONE- *))
          (S-CLR
            (NP-SBJ (-NONE- *))
            (VP
              (TO to)
              (VP
                (VB make)
                (NP (NNP Kent) (NN cigarette) (NNS filters))))))))
    (VP
      (VBZ has)
      (VP
        (VBN caused)
        (NP
          (NP (DT a) (JJ high) (NN percentage))
          (PP (IN of) (NP (NN cancer) (NNS deaths)))
          (PP-LOC
            (IN among)
            (NP
              (NP (DT a) (NN group))
              (PP
                (IN of)
                (NP
                  (NP (NNS workers))
                  (RRC
                    (VP
                      (VBN exposed)
                      (NP (-NONE- *))
                      (PP-CLR (TO to) (NP (PRP it)))
                      (ADVP-TMP
                        (NP
                 

In [14]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents())

# first few tagged sentences
wsj[:1]

[[('Pierre', 'NNP'),
  ('Vinken', 'NNP'),
  (',', ','),
  ('61', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  (',', ','),
  ('will', 'MD'),
  ('join', 'VB'),
  ('the', 'DT'),
  ('board', 'NN'),
  ('as', 'IN'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('Nov.', 'NNP'),
  ('29', 'CD'),
  ('.', '.')]]

#### Prepare corpus for training

For the training purpose we will split the dataset into a `70:30` ratio. 70% of the data for the training set and the rest 30% for the test set.

In [18]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(wsj,test_size=0.3)

print(f"Train test split:  Train = {len(train_set)} Test = {len(test_set)}")

Train test split:  Train = 2739 Test = 1175


In [19]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
print(f"Total tagged words : {len(train_tagged_words)}")

train_tagged_words[:5]

Total tagged words : 70401


[('But', 'CC'),
 ('analysts', 'NNS'),
 ('said', 'VBD'),
 ('0', '-NONE-'),
 ('early', 'JJ')]

#### Create token and tag set

Next, we will create a tokens variable that will contain all the tokens from the train_tagged_words. 

Furthermore, we also need to create a vocabulary and set of all unique tags in the training data.

In [23]:
# tokens
tokens = [pair[0] for pair in train_tagged_words]

# the unique set of the token act as a vocabulary
vocabs = set(tokens)
print(f"Total vocabularies: {len(vocabs)}")

# number of tags
tags = set([pair[1] for pair in train_tagged_words])
print("Total tags: ",len(tags))


Total vocabularies: 10270
Total tags:  46


### Implementation of HMM

**Computation of Emission probabilities**

In [24]:
import numpy as np

In [26]:
# computing P(w/t) and storing in T x V matrix
t = len(tags)
v = len(vocabs)
w_given_t = np.zeros((t, v))
w_given_t.shape

(46, 10270)

In [28]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_ds = train_tagged_words):
    tag_list = [pair for pair in train_ds if pair[1]==tag]
    count_tag = len(tag_list)
    
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]

    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)

In [29]:
# examples
# large
print("\n", "large")
print(word_given_tag('large', 'JJ'))
print(word_given_tag('large', 'VB'))
print(word_given_tag('large', 'NN'), "\n")
# will
print("\n", "will")
print(word_given_tag('will', 'MD'))
print(word_given_tag('will', 'NN'))
print(word_given_tag('will', 'VB'))
# book
print("\n", "book")
print(word_given_tag('book', 'NN'))
print(word_given_tag('book', 'VB'))


 large
(24, 4059)
(0, 1764)
(0, 9150) 


 will
(205, 636)
(1, 9150)
(0, 1764)

 book
(6, 9150)
(1, 1764)


**Computation of Transition Probabilities**

In [30]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [32]:
# examples
print(t2_given_t1(t2='NNP', t1='JJ'))
print(t2_given_t1('NN', 'JJ'))
print(t2_given_t1('NN', 'DT'))
print(t2_given_t1('NNP', 'VB'))
print(t2_given_t1(',', 'NNP'))
print(t2_given_t1('PRP', 'PRP'))
print(t2_given_t1('VBG', 'NNP'))

(133, 4059)
(1800, 4059)
(2656, 5720)
(60, 1764)
(1048, 6689)
(3, 1181)
(5, 6689)


In [33]:
#Please note P(tag|start) is same as P(tag|'.')
print(t2_given_t1('DT', '.'))
print(t2_given_t1('VBG', '.'))
print(t2_given_t1('NN', '.'))
print(t2_given_t1('NNP', '.'))

(576, 2712)
(11, 2712)
(118, 2712)
(497, 2712)


Next, we will create a transition matrix of tags of dimension txt

In [37]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')

for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

tags_matrix

array([[0.00591716, 0.03550296, 0.        , ..., 0.00591716, 0.        ,
        0.        ],
       [0.        , 0.        , 0.00157233, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01123596, 0.        , ..., 0.01123596, 0.        ,
        0.        ],
       ...,
       [0.        , 0.01098901, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00211416, 0.        , ..., 0.00634249, 0.00211416,
        0.00634249]], dtype=float32)

In [41]:
import pandas as pd
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
tags_df

,NNPS,MD,RBR,-NONE-,JJ,$,JJR,``,VBG,RBS,...,WP,WDT,PDT,JJS,RB,IN,#,-LRB-,PRP$,''
NNPS,0.005917,0.035503,0.000000,0.005917,0.000000,0.000000,0.000000,0.005917,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.011834,0.100592,0.000000,0.005917,0.000000,0.000000
MD,0.000000,0.000000,0.001572,0.007862,0.001572,0.000000,0.000000,0.003145,0.001572,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.169811,0.000000,0.000000,0.000000,0.000000,0.000000
RBR,0.000000,0.011236,0.000000,0.033708,0.348315,0.000000,0.000000,0.000000,0.011236,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.112360,0.213483,0.000000,0.011236,0.000000,0.000000
-NONE-,0.000000,0.013761,0.001311,0.072521,0.016601,0.003058,0.001529,0.002840,0.078637,0.000218,...,0.000218,0.000000,0.000000,0.000000,0.022936,0.143731,0.000000,0.001747,0.003277,0.000000
JJ,0.000985,0.000246,0.000493,0.020695,0.065780,0.001725,0.000246,0.001971,0.004188,0.000000,...,0.000246,0.000000,0.000000,0.000246,0.003203,0.055679,0.000000,0.000493,0.000000,0.003449
$,0.000000,0.000000,0.000000,0.000000,0.010121,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
JJR,0.000000,0.000000,0.000000,0.019011,0.057034,0.000000,0.000000,0.000000,0.003802,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003802,0.330798,0.000000,0.000000,0.000000,0.000000
``,0.000000,0.012270,0.000000,0.044990,0.100204,0.000000,0.000000,0.000000,0.004090,0.000000,...,0.008180,0.000000,0.000000,0.000000,0.071575,0.069530,0.000000,0.000000,0.004090,0.000000
VBG,0.000961,0.000000,0.003842,0.079731,0.060519,0.008646,0.016330,0.003842,0.001921,0.000961,...,0.001921,0.000000,0.000961,0.000000,0.039385,0.118156,0.000000,0.000000,0.028818,0.000961
RBS,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000


### Implementation of Viterbi algorithm

The purpose of the Viterbi algorithm is to make an inference based on a trained model and some observed data. 

- It works by asking a question: given the trained parameter matrices and data, what is the choice of states such that the joint probability reaches maximum? 
- In other words, what is the most likely choice given the data and the trained model? This statement can be visualized as the following formula, and obviously, the answer depends on the data!

**Further Reading:**
- [Viterbi](https://medium.com/analytics-vidhya/viterbi-algorithm-for-prediction-with-hmm-part-3-of-the-hmm-series-6466ce2f5dc6)
- Further reading: [slide](https://www.cl.cam.ac.uk/teaching/1718/MLRD/slides/slides9.pdf)


In [42]:
# Viterbi Heuristic
def Viterbi(words, train_ds = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_ds]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

Now, we will tag the test sentences using the Viterbi algorithm

In [47]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/rughimire/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/rughimire/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Inference

In [48]:
sentence_test = 'This is the test sentence to be tagged using HMM'
word_tokens = nltk.word_tokenize(sentence_test)
tagged_seq = Viterbi(word_tokens)

tagged_seq

[('This', 'DT'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('test', 'NN'),
 ('sentence', 'NN'),
 ('to', 'TO'),
 ('be', 'VB'),
 ('tagged', 'NNPS'),
 ('using', 'NNPS'),
 ('HMM', 'NNPS')]

**Documentation**
- Coding:
[Complete Example](https://wisdomml.in/hidden-markov-model-hmm-in-nlp-python/#Viterbi_Algorithm-2)
- [A deep dive into part-of-speech tagging using the Viterbi algorithm](https://www.freecodecamp.org/news/a-deep-dive-into-part-of-speech-tagging-using-viterbi-algorithm-17c8de32e8bc/)